In [1]:
import pandas as pd
import numpy as np
import datetime
import importlib
import pickle

import data_utils
import model_utils
import train_utils
import evaluation
import submission
importlib.reload(data_utils)
importlib.reload(model_utils)
importlib.reload(train_utils)
importlib.reload(evaluation)
importlib.reload(submission)

import torch
import torch.nn as nn
from torchtext.data import TabularDataset, Field, RawField, BucketIterator, Iterator
from transformers import BertForSequenceClassification, BertTokenizer, BertModel

I0211 20:28:30.347209 139670379685696 file_utils.py:38] PyTorch version 1.4.0 available.


### Training 

In [2]:
train_data, val_data, test_data = data_utils.prep_all_data(
    train_file="train_real.csv", val_file="val.csv", test_file="test.csv", batch_size=32)

In [3]:
m = model_utils.BertMovie()

I0211 20:28:43.068177 139670379685696 tokenization_utils.py:418] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-vocab.txt from cache at /home/molly/.cache/torch/transformers/9b3c03a36e83b13d5ba95ac965c9f9074a99e14340c523ab405703179e79fc46.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
I0211 20:28:43.542003 139670379685696 configuration_utils.py:254] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-config.json from cache at /home/molly/.cache/torch/transformers/6dfaed860471b03ab5b9acb6153bea82b6632fb9bbe514d3fff050fe1319ee6d.fc076a4d5f1edf25ea3a2bd66e9f6f295dcd64c81dfef5b3f5a3eb2a82751ad1
I0211 20:28:43.543458 139670379685696 configuration_utils.py:290] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "do_sample": false,
  "eos_token_ids": 0,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hid

In [7]:
result = train_utils.train(train_data, val_data, m,
                          lr=1e-5, print_freq=1, max_epoch=100)

Epoch: 0, LR: 1e-05, Train Loss: 575.9486, Val Loss: 294.2758, Val f1 0.126
Epoch: 1, LR: 1e-05, Train Loss: 254.9178, Val Loss: 200.7394, Val f1 0.126
Epoch: 2, LR: 1e-05, Train Loss: 194.9430, Val Loss: 169.1680, Val f1 0.126
Epoch: 3, LR: 1e-05, Train Loss: 171.5428, Val Loss: 154.6524, Val f1 0.126
Epoch: 4, LR: 1e-05, Train Loss: 159.7396, Val Loss: 146.6598, Val f1 0.126
Epoch: 5, LR: 1e-05, Train Loss: 152.4854, Val Loss: 140.9484, Val f1 0.126
Epoch: 6, LR: 1e-05, Train Loss: 145.5534, Val Loss: 133.2889, Val f1 0.138
Epoch: 7, LR: 1e-05, Train Loss: 135.8025, Val Loss: 121.8225, Val f1 0.194
Epoch: 8, LR: 1e-05, Train Loss: 122.2849, Val Loss: 107.0806, Val f1 0.311
Epoch: 9, LR: 1e-05, Train Loss: 107.3720, Val Loss: 93.5628, Val f1 0.511
Epoch: 10, LR: 1e-05, Train Loss: 93.7622, Val Loss: 83.2128, Val f1 0.592
Epoch: 11, LR: 1e-05, Train Loss: 82.6304, Val Loss: 74.1128, Val f1 0.663
Epoch: 12, LR: 1e-05, Train Loss: 73.2444, Val Loss: 67.7242, Val f1 0.714
Epoch: 13, LR: 1

In [8]:
torch.save(result["trained_model"], "./data/model_checkpoints/bert_large_feb05.mdl")

/home/molly/anaconda3/lib/python3.6/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type BertMovie. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


### Ensemble

In [12]:
train_data, val_data, test_data = data_utils.prep_all_data(
    train_file="train_real.csv", val_file="val.csv", test_file="test.csv")
train_data_phrase, val_data_phrase, test_data_phrase = data_utils.prep_all_data(
    path="./data/movie_name_condensed_data/", 
    train_file="train_real.csv", val_file="val.csv", test_file="test.csv")

m_gram = torch.load("./data/model_checkpoints/ngram_MLP_Jan31.mdl")
m_gram_phrase = torch.load("./data/model_checkpoints/ngram_phrase_MLP_Jan31.mdl")
m_GRU = torch.load("./data/model_checkpoints/GRU_Jan31.mdl")
m_GRU_phrase = torch.load("./data/model_checkpoints/GRU_phrase_Jan31.mdl")
m_bert = result["trained_model"]

/home/molly/anaconda3/lib/python3.6/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'model_utils.GRU' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [187]:
importlib.reload(submission)

ensemble = submission.Ensemble(models=[m_gram, m_gram_phrase], 
                               val_data=[val_data, val_data_phrase],
                               test_data=[test_data, test_data_phrase])
df_val = ensemble.get_ensemble_result(submission=False)

ensemble f1 val score: 0.818118118118118


In [16]:
importlib.reload(submission)

ensemble = submission.Ensemble(models=[m_bert, m_gram, m_gram_phrase], 
                               val_data=[val_data, val_data, val_data_phrase],
                               test_data=[test_data, test_data, test_data_phrase])
df_val = ensemble.get_ensemble_result(submission=False)

ensemble f1 val score: 0.8632632632632633


In [17]:
importlib.reload(submission)

ensemble = submission.Ensemble(models=[m_bert, m_gram_phrase], 
                               val_data=[val_data, val_data_phrase],
                               test_data=[test_data, test_data_phrase])
df_val = ensemble.get_ensemble_result(submission=False)

ensemble f1 val score: 0.8922922922922921


In [19]:
importlib.reload(submission)

ensemble = submission.Ensemble(models=[m_bert], 
                               val_data=[val_data],
                               test_data=[test_data])
df_val = ensemble.get_ensemble_result(submission=False)

ensemble f1 val score: 0.8896896896896896


### Generate submission file for kaggle 

In [18]:
df = ensemble.get_ensemble_result(submission=True)
ensemble.save_submission_file("bert-large+ngram-phrase")

### Error analysis for best model

In [20]:
errors = df_val[df_val["CORE RELATIONS"]!=df_val.raw_label]; errors.shape
correct = df_val[df_val["CORE RELATIONS"]==df_val.raw_label]; errors.shape

(53, 3)

In [21]:
accuracy = 1-len(errors)/len(df_val); accuracy

0.8408408408408409

In [22]:
errors.raw_label.value_counts()

movie_other                                                8
other                                                      8
NO_REL                                                     4
movie.rating movie.initial_release_date                    3
movie.directed_by                                          3
movie.starring.actor actor.gender                          3
movie.starring.actor movie.starring.character              2
movie.star_rating                                          2
movie.rating                                               2
movie.genre movie.initial_release_date                     1
movie.country movie.genre                                  1
movie.locations movie.country                              1
movie.country                                              1
movie.genre movie.description                              1
movie.country movie.subjects                               1
movie.rating movie.subjects                                1
movie.locations movie.in

In [23]:
errors[errors.raw_label.isin(["other", "movie_other"])]

,CORE RELATIONS,text,raw_label
ID,,,
2861,movie.showing,where can i rent this movie,movie_other
2896,movie.genre,search for dramas like downton abbey,movie_other
2904,movie.subjects,what other movies are about the holocaust simi...,movie_other
2914,movie.subjects,after watching this movie will i have nightmar...,movie_other
2922,movie.language,what do the actors speak,other
2939,movie.genre,scary,other
2943,movie.subjects,zombies,other
2951,movie.country movie.language,arabic life of a movie,other
2964,movie.directed_by,did you have fun directing gilda mister vidor,movie_other


In [205]:
correct[correct.raw_label.isin(["other", "movie_other"])]

,CORE RELATIONS,text,raw_label
ID,,,
2844,movie_other,find movie based on criteria,movie_other
2855,movie_other,search by stars,movie_other
2857,movie_other,searching for cheap movies,movie_other
2861,movie_other,where can i rent this movie,movie_other
2889,movie_other,middle aged white actors,movie_other
2894,movie_other,movies like in july,movie_other
2922,other,what do the actors speak,other
2939,other,scary,other
2954,other,can i have a map of britain please,other


In [350]:
df_train = pd.read_csv("./data/train_real.csv", index_col="ID")[["text", "raw_label"]]
df_test = pd.read_csv("./data/holdout_test.csv", index_col="ID")[["text", "raw_label"]]
df_val = pd.read_csv("./data/val.csv", index_col="ID")[["text", "raw_label"]]
df_all_train = pd.read_csv("./data/train.csv", index_col="ID")[["text", "raw_label"]]

In [351]:
df_train[df_train.raw_label.isin(["NO_REL", "other"])]

,text,raw_label
ID,,
735,i need movie information for life is beautiful,NO_REL
737,i want to know more about the movie life is be...,NO_REL
738,i would like info on the movie life is beautiful,NO_REL
739,please show me info on the movie life is beaut...,NO_REL
740,search for the movie life is beautiful,NO_REL
742,can i see more info the movie life is beautiful,NO_REL
743,can you show me info on the movie life is beau...,NO_REL
744,display information for the movie life is beau...,NO_REL
773,i am looking for the information on the movie ...,NO_REL


In [265]:
df_val.index.value_counts().shape

(315,)

In [266]:
df_train.index.value_counts().shape

(2702,)

In [268]:
df_all_train.index.value_counts().shape

(3338,)

In [269]:
321 + 315 + 2702

3338

In [271]:
df_val.shape

(333, 2)